In [3]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

In [4]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec
# gera uma lista de tupla com as possibilidades dos codigos de serventias e um intervalo dos anos
servs_anos = list(itertools.product(list(serventias_jec.cod_serv),range(2013,2017)))

# gera a lista dos arquivos que deverão ser lidos
processos_serv_ano = [TJData.processo(serv,ano) for serv,ano in servs_anos]

# colunas que serão utilizadas no processo
usecols=['COD_PROC','COD_COMP','COD_SERV','ID_PROC','COD_ASSUNTO','DATA_CAD']

files_available = TJData.list_files_in_serventias(TJData.processo(None,None),servs_anos)
servs_anos = list(map(lambda x : TJData.extract_serventia_ano_tuple(x),files_available))
all_dfs = []
for serv,ano in servs_anos:
    df = TJData.read_csv(TJData.processo(serv,ano),usecols=usecols)
    df = df.rename(columns=lambda x : x.lower())
    df = df.rename(columns={'cod_comp':'competencia','cod_serv':'serventia','cod_assunto':'assunto'})
    df.cod_proc = TJData.to_utf8_bytes(df.cod_proc)
    df.data_cad = TJData.to_datetime_iso_format(df.data_cad)
    df.data_cad = TJData.to_utf8_bytes(df.data_cad)
    all_dfs.append(df)
processos = TJData.to_json(pd.concat(all_dfs))
processos.shape

AttributeError: 'list' object has no attribute 'shape'

In [9]:
len(processos)

153009